# Lambda calculus in Kotlin

## Background knowledge

Lambda calculus is a formal system for expressing computation based on function abstraction and application.
I assume you already know what a function is in math and programming. Then Let's start from the basics.

In the world of lambda calculus, functions are anonymous, taking a single parameter and returning a single value.
Even more, everythings in lambda calculus is a function.

Let's define a function:

```
λx.x
```

It means `x` is a parameter and `x` is a return value. This function takes a parameter `x` and returns `x` without any difference.

We should remember everything in lambda calculus is a function. We can define some other functions:

```
λx.x(x)(x)
λx.λy.x
```

Confused? The first function's parameter is `x`, which is simple. Because `x` is a function, so `x(x)` means passing `x` itself as a parameter to `x`. Furthermore, `x(x)` is also a function, so `x(x)(x)` means passing `x` as a parameter to `x(x)`.
The second function's parameter is `x`. It returns a expression `λy.x`, which is also a function.

To make it simpler to write, the example above is often written in:

```
λx.xxx
λxy.x
```

Let me explain it: `abc` means `(a(b))(c)`, just like `1+2+3` means `((1+2)+3)`.
`λabc.xyz` means `λa.λb.λc.xyz`. When it takes a parameter `a`, it returns `λb.λc.xyz`, which is also a function.
And the returned function takes a parameter `b` and returns `λc.xyz`.
Finally, the returned function takes a parameter `c` and returns `xyz`.

After these practices, you can conclude that there are two things in lambda calculus: function and applying.

## Basic interface definition

Since `kotlin.FunctionN` is not able to express a recursive lambda type, whose parameter and return type are both itself, we define a new interface `Lambda` to express it.

In [1]:
fun interface Lambda {
    operator fun invoke(f: Lambda): Lambda
}

## Shorthand for defining a lambda expression with currying

If we want to write `λab.a` in Kotlin, embedded braces like `{ a -> { b -> a } }` are annoying.
Thus, we can use currying to simplify it.

In [2]:
fun lambda(block: (Lambda) -> Lambda) =
    Lambda { p0 -> block(p0) }

fun lambda(block: (Lambda, Lambda) -> Lambda) =
    Lambda { p0 -> lambda { p1 -> block(p0, p1) } }

fun lambda(block: (Lambda, Lambda, Lambda) -> Lambda) =
    Lambda { p0 -> lambda { p1, p2 -> block(p0, p1, p2) } }

fun lambda(block: (Lambda, Lambda, Lambda, Lambda) -> Lambda) =
    Lambda { p0 -> lambda { p1, p2, p3 -> block(p0, p1, p2, p3) } }

fun lambda(block: (Lambda, Lambda, Lambda, Lambda, Lambda) -> Lambda) =
    Lambda { p0 -> lambda { p1, p2, p3, p4 -> block(p0, p1, p2, p3, p4) } }

fun lambda(block: (Lambda, Lambda, Lambda, Lambda, Lambda, Lambda) -> Lambda) =
    Lambda { p0 -> lambda { p1, p2, p3, p4, p5 -> block(p0, p1, p2, p3, p4, p5) } }

## Church numbers

Then we can define Church numbers.

Church number is a way to define natural numbers.
It is a function `n` that takes a function `f` and a value `x` and apply `f` for `n` times on `x`.
For example:

In [3]:
val zero = lambda { f, x -> x }
val one = lambda { f, x -> f(x) }
val two = lambda { f, x -> f(f(x)) }
val three = lambda { f, x -> f(f(f(x))) }
val four = lambda { f, x -> f(f(f(f(x)))) }
val five = lambda { f, x -> f(f(f(f(f(x))))) }

To make it visible, we can define a function to transform from Church numbers to `kotlin.Int`.

In [4]:
val identity = lambda { it -> it }

fun Lambda.toInt(): Int {
    var count = 0
    this { count++; it }(identity)
    return count
}

And the reversed transform:

In [5]:
fun Int.toChurch(): Lambda =
    if (this <= 0) zero
    else Lambda { f -> Lambda { x -> f((this - 1).toChurch()(f)(x)) } }

Let's see the output:

In [6]:
println(zero.toInt())
println(one.toInt())
println(two.toInt())
println(0.toChurch().toInt())
println(100.toChurch().toInt())

0
1
2
0
100


It matches our expectation.

## Basic arithmetics

Let's start from the simplest ones.

In [7]:
val successor = lambda { n, f, x -> f(n(f)(x)) }
val multiply = lambda { m, n, f -> m(n(f)) }
val pow = lambda { a, b -> b(a) }

And let's define operators in Kotlin for them.

In [8]:
operator fun Lambda.plus(other: Lambda): Lambda = this(successor)(other)
operator fun Lambda.times(other: Lambda): Lambda = multiply(this)(other)

Let's test them.

In [9]:
println((one + two * three).toInt())

7


When it comes to substraction, things are bit of more complex.

We need a pair `(0, 0)`. Then we define a `Φ` combinator, which moves a pair `(a, b)` to `(b, b+1)`.
We invoke `Φ` for `n` times on `(0, 0)`, and we get `(n-1, n)` in the end.
Finally, we only need to get the first element of the pair. Then we get `n-1` from `n`.

In [10]:
val pair = lambda { a, b, f -> f(a)(b) }
val first = lambda { a, b -> a }
val second = lambda { a, b -> b }
val fai = lambda { p -> pair(p(second))(successor(p(second))) }

Let's test out pair.

In [11]:
{
    val myPair = pair(10.toChurch())(20.toChurch())
    println(myPair(first).toInt())
    println(myPair(second).toInt())
    println(fai(myPair)(first).toInt())
    println(fai(myPair)(second).toInt())
}()

10
20
20
21


It seems that `pair` and `fai` works correctly.

Now we can write the substraction.

In [12]:
val predecessor = lambda { n -> n(fai)(pair(zero)(zero))(first) }
val subtract = lambda { a, b -> b(predecessor)(a) }
operator fun Lambda.minus(other: Lambda): Lambda = subtract(this)(other)

Let's test it.

In [13]:
println((25.toChurch() - 13.toChurch()).toInt())

12


## Boolean arithmetics

We need to define `true` and `false` first.

In [14]:
val `true` = lambda { a, b -> a }
val `false` = lambda { a, b -> b }

Have you noticed? `true` is same as `first` of `pair` and `false` is same as `second` of `pair`.

Then, we can define basic logic operators.

In [15]:
val and = lambda { a, b -> a(b)(a) }
val or = lambda { a, b -> a(a)(b) }
val not = lambda { a -> a(`false`)(`true`) }
val xor = lambda { a, b -> a(not(b))(b) }

Confused? Me too.

They are kind of tricky things, so we need to check if they are right. Make it visible!

In [16]:
fun Lambda.toBoolean(): Boolean {
    var result = false
    this(lambda { it -> result = true;it })(lambda { it -> result = false;it })(identity)
    return result
}

In [17]:
println("--- NOT ---")
println("not(true) = ${not(`true`).toBoolean()}")
println("not(false) = ${not(`false`).toBoolean()}")
println()

println("--- AND ---")
println("true and true = ${and(`true`)(`true`).toBoolean()}")
println("true and false = ${and(`true`)(`false`).toBoolean()}")
println("false and true = ${and(`false`)(`true`).toBoolean()}")
println("false and false = ${and(`false`)(`false`).toBoolean()}")
println()

println("--- OR ---")
println("true or true = ${or(`true`)(`true`).toBoolean()}")
println("true or false = ${or(`true`)(`false`).toBoolean()}")
println("false or true = ${or(`false`)(`true`).toBoolean()}")
println("false or false = ${or(`false`)(`false`).toBoolean()}")
println()

println("--- XOR ---")
println("true xor true = ${xor(`true`)(`true`).toBoolean()}")
println("true xor false = ${xor(`true`)(`false`).toBoolean()}")
println("false xor true = ${xor(`false`)(`true`).toBoolean()}")
println("false xor false = ${xor(`false`)(`false`).toBoolean()}")

--- NOT ---
not(true) = false
not(false) = true

--- AND ---
true and true = true
true and false = false
false and true = false
false and false = false

--- OR ---
true or true = true
true or false = true
false or true = true
false or false = false

--- XOR ---
true xor true = false
true xor false = true
false xor true = true
false xor false = false


After boolean calculations, do you want `if` and `else`? Let's do it!

In [18]:
val `if` = lambda { c, t, f -> c(t)(f) }

In [19]:
println("if (true) 25 else 10 = ${`if`(`true`)(25.toChurch())(10.toChurch()).toInt()}")
println("if (false) 25 else 10 = ${`if`(`false`)(25.toChurch())(10.toChurch()).toInt()}")

if (true) 25 else 10 = 25
if (false) 25 else 10 = 10


Since `zero` is `λfx.x`, we can easily define `isZero` and comparators.

Please notice that in Church numbers, there's no negative number. `5-10` is `0`.

In [20]:
val isZero = lambda { n -> n(`true`(`false`))(`true`) }
val le = lambda { a, b -> isZero(a - b) }
val eq = lambda { a, b -> and(le(a)(b))(le(b)(a)) }

In [21]:
println("--- isZero ---")
println("isZero(zero) = ${isZero(zero).toBoolean()}")
println("isZero(three) = ${isZero(three).toBoolean()}")
println("--- Less than or Equal (le) ---")
println("2 <= 5 = ${le(two)(five).toBoolean()}")
println("5 <= 2 = ${le(five)(two).toBoolean()}")
println("3 <= 3 = ${le(three)(three).toBoolean()}")
println("--- Equal (eq) ---")
println("3 == 5 = ${eq(three)(five).toBoolean()}")
println("5 == 3 = ${eq(five)(three).toBoolean()}")
println("3 == 3 = ${eq(three)(three).toBoolean()}")

--- isZero ---
isZero(zero) = true
isZero(three) = false
--- Less than or Equal (le) ---
2 <= 5 = true
5 <= 2 = false
3 <= 3 = true
--- Equal (eq) ---
3 == 5 = false
5 == 3 = false
3 == 3 = true


## Recursive

After mastering these basic operations, it's time to try some high-level things!

Let's define a recursive `fibonacci` function.

In the world of lambda calculus, every function is anonymous, so we cannot refer to it by name.
Luckily, we have parameters. We can pass the lambda it self as a parameter.
Thus, we can define a recursive `fibonacci` function that takes 2 parameters: `self` and `n`.
Please notice that `self` is also a lambda that need `self` as a parameter.

In [22]:
val metaFibonacci0 =
    lambda { self, n -> if (le(n)(two).toBoolean()) one else self(self)(n - one) + self(self)(n - two) }
val fibonacci0 = metaFibonacci0(metaFibonacci0)

By this interesting trick, we can define a recursive function in lambda calculus.

In [23]:
for (n in 1..10) {
    println("fibonacci($n) = ${fibonacci0(n.toChurch()).toInt()}")
}

fibonacci(1) = 1
fibonacci(2) = 1
fibonacci(3) = 2
fibonacci(4) = 3
fibonacci(5) = 5
fibonacci(6) = 8
fibonacci(7) = 13
fibonacci(8) = 21
fibonacci(9) = 34
fibonacci(10) = 55


We noticed some repeats in our code (`self(self)` and `meta(meta)`). Let's wipe them out.

In [24]:
val omega = lambda { r -> r(r) } // this is called K combinator or Ω-combinator
val metaFibonacci1 = lambda { self, n ->
    val f = omega(self)
    if(le(n)(two).toBoolean()) one else f(n - one) + f(n - two)
}
val fibonacci1 = omega(metaFibonacci1)

In [25]:
for (n in 1..10) {
    println("fibonacci($n) = ${fibonacci1(n.toChurch()).toInt()}")
}

fibonacci(1) = 1
fibonacci(2) = 1
fibonacci(3) = 2
fibonacci(4) = 3
fibonacci(5) = 5
fibonacci(6) = 8
fibonacci(7) = 13
fibonacci(8) = 21
fibonacci(9) = 34
fibonacci(10) = 55


But it's still repeating. Let's do some more.

We want a simpler `metaFibonacci` that can be written as: `λfx.if(x<=2) 1 else f(x-1)+f(x-2)`. Then we need to find a way to make a infinite call like `f(f(f(f(f(...))))`.

We can do it by using `Y combinator`. It has a interesting property: `Y(f) = f(Y(f)) = f(f(Y(f)) = f(f(f(f(f(...))))`.

Without knowing how we can find it, we can test it with a fake one.

In [26]:
val metaFibonacci2 =
    lambda { f, n -> if (le(n)(two).toBoolean()) one else f(n - one) + f(n - two) }
val fakeY = lambda { f -> f(f(f(f(f(f(f(f))))))) } // fake Y combinator, limited to 8 levels
val fakefibonacci2 = fakeY(metaFibonacci2)

In [27]:
for (n in 1..10) {
    println("fibonacci($n) = ${fakefibonacci2(n.toChurch()).toInt()}")
}

fibonacci(1) = 1
fibonacci(2) = 1
fibonacci(3) = 2
fibonacci(4) = 3
fibonacci(5) = 5
fibonacci(6) = 8
fibonacci(7) = 13
fibonacci(8) = 21
fibonacci(9) = 32
fibonacci(10) = 40


fYou can see things going wrong after `fibonacci(8)` because it's not an infinite `f(f(f(f(f(...))))`.

To define a real `Y` combinator, we need to use the trick used in `fibonacci0`.
If we can't really write `Y = λf.f(Yf)` because we can't refer to `y` by name, we can refer it as a parameter.

In [28]:
val metaY = lambda { self, f -> f(self(self)(f)) }
val y = metaY(metaY)

You can expand this `Y` combinator:
```
Y = metaY(metaY)
  = (λab.b((aab))(λab.b(aab)) // Y
  = λf.f((λab.b(a(a))(λab.b(a(a)))f) // λf.f(Yf)
```
It has a more commonly known syntax: `λf.(λx.f(xx))(λx.f(xx))`.

In [29]:
try {
    val fibonacci2 = y(metaFibonacci2)
    fibonacci2(3.toChurch())
} catch (e: StackOverflowError) {
    println("Stack overflow!")
}


Stack overflow!


But when you want to use this `Y` combinator to build an actual `fibonacci` function, you will fail from `StackOverflowError`.
The reason is that we have an infinite `f(f(f(f(f(...))))`.

To solve this issue, we can use the trick of deferred evaluation, or you can call it a functional parameter.
This solution is called `Z` combinator.

In [30]:
val metaZ = lambda { self, f -> f { x -> self(self)(f)(x) } }
val z = metaZ(metaZ)
val fibonacci2 = z(metaFibonacci2)

In [31]:
for (n in 1..10) {
    println("fibonacci($n) = ${fibonacci2(n.toChurch()).toInt()}")
}

fibonacci(1) = 1
fibonacci(2) = 1
fibonacci(3) = 2
fibonacci(4) = 3
fibonacci(5) = 5
fibonacci(6) = 8
fibonacci(7) = 13
fibonacci(8) = 21
fibonacci(9) = 34
fibonacci(10) = 55


## References

- [Wikipedia](https://en.wikipedia.org/wiki/Lambda_calculus)
- [A Tutorial Introduction to the Lambda Calculus](https://personal.utdallas.edu/~gupta/courses/apl/lambda.pdf)